# Data PreProcessing

### Dealing with missing values

In [1]:
import pandas as pd
from pandas import DataFrame

In [3]:
from io import StringIO # String IO formats strings
csv_data = '''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0'''
csv_data = unicode(csv_data)
df = pd.read_csv(StringIO(csv_data)) # StringIO allows us to read string assigned to csv_data into a DF as if it were
# a regular CSV
df

,A,B,C,D
0,1,2,3,4
1,5,6,NaN,8
2,10,11,12,NaN


In [61]:
# we can use the .isnull method to return a DataFrame with boolean values that indicate whether a cell contains a
# numeric value (False) or if data is missing (True)
# we can sum up the null values in each column using sum
print df.isnull().sum(axis = 0) # calculate sums row wise - agg data will be represented by column
print df.isnull().sum(axis = 1) # calculate sums column wise - agg data will be represented by rows

A    0
B    0
C    1
D    1
dtype: int64
0    0
1    1
2    1
dtype: int64


#### Eliminating samples or features with missing values

In [4]:
# remove rows with dropna()
# df.dropna() by default drops rows with nas in them
# remove features with dropna(axis=1)
df.dropna(axis = 1) # drop columns column wise - if any NAs then drop column

,A,B
0,1,2
1,5,6
2,10,11


In [28]:
# only drop rows where all columns are NaN
df.dropna(how= 'all')
# drop rows that have not at least 4 non-NaN values
df.dropna(thresh=4)
# only drop rows where NaN appear in specific columns (here: 'C')
df.dropna(subset=['C'])



,A,B,C,D
0,1,2,3,4
2,10,11,12,NaN


## Imputing Missing Values

In [4]:
csv_data = '''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0'''
csv_data = unicode(csv_data)
df = pd.read_csv(StringIO(csv_data)) # StringIO allows us to read string assigned to csv_data into a DF as if it were
# a regular CSV
df

,A,B,C,D
0,1,2,3,4
1,5,6,NaN,8
2,10,11,12,NaN


In [5]:
from sklearn.preprocessing import Imputer
imr = Imputer(missing_values='NaN', strategy ='mean', axis = 0)
imr = imr.fit(df)
imputed_data = imr.transform(df.values)
imputed_data

array([[  1. ,   2. ,   3. ,   4. ],
       [  5. ,   6. ,   7.5,   8. ],
       [ 10. ,  11. ,  12. ,   6. ]])

In [6]:
df2 = pd.DataFrame(imputed_data, columns = ['A','B','C','D'])

In [7]:
df2

,A,B,C,D
0,1,2,3.0,4
1,5,6,7.5,8
2,10,11,12.0,6


## Mapping Ordinal Features

In [8]:
import pandas as pd
import numpy as np

df = pd.DataFrame([['green', 'M', 10.1, 'class1'],
                  ['red', 'L', 13.5, 'class2'],
                  ['blue', 'XL', 15.3, 'class1']], 
                  columns = ['color','size','price','classlabel'])
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


In [11]:
print 'unique values are: ', np.unique(df['size'].values)
# create a dict mapping for string values to integers
size_mapping = {'XL':3, 'L':2, 'M':1}

unique values are:  ['L' 'M' 'XL']


In [13]:
# .map is a method that takes each value in an array and maps it to a lookup such as the dict just defined
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [14]:
inv_size_mapping = {v:k for k,v in size_mapping.items()} # use a dict comprehension to create reverse mapping
df['size'] = df['size'].map(inv_size_mapping) # re-map the reverse mapping back to df['size']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


## Mapping Class Variables

In [15]:
# lets use another dictionary comprehension to do this manually
class_mapping = {label:idx for idx, label in enumerate(np.unique(df['classlabel']))} # make sure to use np.unique
class_mapping

{'class1': 0, 'class2': 1}

In [16]:
# now replace the classlabel feature with the new integer mapped class label feature
df['classlabel'] = df['classlabel'].map(class_mapping)

In [17]:
df

,color,size,price,classlabel
0,green,M,10.1,0
1,red,L,13.5,1
2,blue,XL,15.3,0


### Encoding Class Labels With SciKit Learn

In [19]:
df = pd.DataFrame([['green', 'M', 10.1, 'class1'],
                  ['red', 'L', 13.5, 'class2'],
                  ['blue', 'XL', 15.3, 'class1']], 
                  columns = ['color','size','price','classlabel'])
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


In [20]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
df['classlabel'] = class_le.fit_transform(df['classlabel']) # fit_transform is a shortcut for calling fit and transform
print df
# separately, and we can use inverse_transform method to transform the int class labels back to original string rep
df['classlabel'] = class_le.inverse_transform(df['classlabel'])
df

   color size  price  classlabel
0  green    M   10.1           0
1    red    L   13.5           1
2   blue   XL   15.3           0


,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


### Handling Nominal Features With One-Hot Encoding

In [42]:
from sklearn.preprocessing import OneHotEncoder


df = pd.DataFrame([['green', 'M', 10.1, 'class1'],
                  ['red', 'L', 13.5, 'class2'],
                  ['blue', 'XL', 15.3, 'class1']], 
                  columns = ['color','size','price','classlabel'])
#df.values
#df[['blue','green','red']] = pd.get_dummies(df['color'])

#pd.get_dummies(df[['price','color']])
# the get_dummies method wil only convert string columns and leave all other columns unchanged
int_map = {v:k for k,v in enumerate(df['color'])}
df['color'] = df['color'].map(int_map)

df[['blue','green','red']] = pd.get_dummies(df['color'])
df
#ohe = OneHotEncoder(categorical_features=[0])
#test = ohe.fit_transform(df['color'].values).toarray()
#test

,color,size,price,classlabel,blue,green,red
0,0,M,10.1,class1,1,0,0
1,1,L,13.5,class2,0,1,0
2,2,XL,15.3,class1,0,0,1


## Partitioning Dataset Into Training Using Wine Data
##### Look at the chemical makeup of 178 wine samples

In [43]:
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None)
df_wine.columns = ['class_label','alcohol','malic_acid','ash','alcalinity_of_ash','magnesium','total_phenols',
                  'flavanoids','nonflavanoid_phenols','proanthocyanine','color_intensity','hue','ODQ280/OD315',
                  'proline']
df_wine.head()

,class_label,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanine,color_intensity,hue,ODQ280/OD315,proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [93]:
df_wine.describe()

,class_label,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanine,color_intensity,hue,ODQ280/OD315,proline
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,1.938202,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258
std,0.775035,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474
min,1.000000,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000
25%,1.000000,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000
50%,2.000000,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000
75%,3.000000,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000
max,3.000000,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000


In [44]:
# split up the data into training and test sets
# separate out class labels
from sklearn.cross_validation import train_test_split
x, y = df_wine.iloc[:,1:].values, df_wine.iloc[:,0].values
# split @ 30% for training set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

## Bringing Features Onto The Same Scale

In [51]:
# we can use the minmax scaler from sklearn or do it ourselves if we want to
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
x_train_norm = mms.fit_transform(x_train)
x_test_norm = mms.transform(x_test) # apply same fit to the test set (always the case here)
x_train_norm[:5,0]

array([ 0.72043011,  0.31989247,  0.60215054,  0.57258065,  0.76075269])

In [54]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_std = scaler.fit_transform(x_train)
x_test_std = scaler.transform(x_test)
x_train_std[:5,0] # take a look at first 5 records for first feature

array([ 0.91083058, -0.95609928,  0.35952243,  0.22169539,  1.09877654])

## Selecting Meaningful Features

In [72]:
# for regularlized models in sklearn, we can simply set the penalty to '11' to yield a sparse L1 solution
# lets train a logistic regression model with an L1 penalty to eliminate the non-important weights of variables
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty='l1',C=0.1)
lr.fit(x_train_std, y_train)
print 'Training accuracy: ', lr.score(x_train_std, y_train)
print 'Test Accuracy: ', lr.score(x_test_std, y_test)
# these models indicate no overfitting, the test accuracy is BETTER than training data set

Training accuracy:  0.983870967742
Test Accuracy:  0.981481481481
